## Investment Case Group Project

In [2]:
import pandas as pd
import numpy as np
dirpath = 'C:/Users/ankit.bhatia/Documents/GitHub/PythonScripts/data/spark_funds/'
file1 = 'companies.txt'
file2 = 'rounds2.csv'
file3 = 'mapping.csv'

In [3]:
companies = pd.read_csv(dirpath+file1,sep='\t',encoding='iso-8859-1')
rounds2 = pd.read_csv(dirpath+file2,encoding='iso-8859-1') # Use 'iso-8859-1' for accented characters

In [4]:
companies.head(3)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN


In [5]:
rounds2.head(3)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0


### Table 1.1
#### Q1. How many unique companies are present in rounds2?

In [6]:
# Normal Count
#rounds2.company_permalink.count()

# Unique count which excludes NA as well by default
rounds2.company_permalink.nunique()

90247

#### Q2. How many unique companies are present in companies?

In [7]:
companies.permalink.nunique()

66368

#### Q3. In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.

In [8]:
# Ist Method
#nunique() will give unique values for each column, column with maximum unique counts can be considered as unique column#### 
companies.nunique().sort_values(ascending=False).iloc[0:1]

# IInd Method
# Companies.describe() will give count, unique, top etc information, we store count & unique values in a temp df and
# then subtracting unique values(1) from count(0) and giving the column name where difference is 0
#companies.describe().iloc[0:2]


permalink    66368
dtype: int64

#### Q4. Are there any companies in the rounds2 file which are not  present in companies ?

In [9]:
# Check the existacce of a column of dataframe1 in column1 of dataframe2
rounds2[rounds2['company_permalink'].str.lower().isin(companies['permalink'].str.lower())== False]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
29597,/ORGANIZATION/E-CÃBICA,/funding-round/8491f74869e4fe8ba9c378394f8fbdea,seed,NaN,01-02-2015,NaN
31863,/ORGANIZATION/ENERGYSTONE-GAMES-ÇµÇ³Æ¸¸Æ,/funding-round/b89553f3d2279c5683ae93f45a21cfe0,seed,NaN,09-08-2014,NaN
45176,/organization/huizuche-com-æ ç§ÿè½¦,/funding-round/8f8a32dbeeb0f831a78702f83af78a36,seed,NaN,18-09-2014,NaN
58473,/ORGANIZATION/MAGNET-TECH-Ç£Ç³Ç§Æ,/funding-round/8fc91fbb32bc95e97f151dd0cb4166bf,seed,NaN,16-08-2014,1625585.0
101036,/organization/tipcat-interactive-æ²èÿä¿¡æ¯ç...,/funding-round/41005928a1439cb2d706a43cb661f60f,seed,NaN,06-09-2010,NaN
109969,/ORGANIZATION/WEICHE-TECH-ÅÈ½¦Ç§Æ,/funding-round/f74e457f838b81fa0b29649740f186d8,venture,A,06-09-2015,NaN
113839,/ORGANIZATION/ZENGAME-Ç¦ Æ¸¸Ç§Æ,/funding-round/6ba28fb4f3eadf5a9c6c81bc5dde6cdf,seed,NaN,17-07-2010,NaN


#### Q5. Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame ?

In [10]:
# lower() the key columns of both dataframes, so they can match in case of diffence in case.
companies['permalink'] = companies['permalink'].str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
master_frame = pd.merge(left = companies,right = rounds2, how ='inner', left_on='permalink',right_on='company_permalink')
master_frame.shape[0]

# From here we will work with master_frame only

114942

### Table 2.1 ( Average Values of Investments for Each of these Funding Types)
#### Q1/2/3/4. Average funding amount of different funding type ?

In [11]:
def avg_funding_amount(funding_type):
    return master_frame[master_frame['funding_round_type']==funding_type]['raised_amount_usd'].mean()

print('venture' ,avg_funding_amount('venture'))
print('angel' ,avg_funding_amount('angel'))
print('private_equity' ,avg_funding_amount('private_equity'))

venture 11748949.129489528
angel 958694.4697530865
private_equity 73308593.02944215


#### Q5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?

In [14]:
## [['']] double brackets are used to keep the dataframe , if we use [''], it will be converted into series
most_suitable = master_frame.groupby(by='funding_round_type')[['raised_amount_usd']].mean()
most_suitable = most_suitable[(most_suitable.raised_amount_usd >= 5000000) & (most_suitable.raised_amount_usd <= 15000000)]
most_suitable

,raised_amount_usd
funding_round_type,
venture,1.174895e+07


In [15]:
# Filter the master_frame dataset based on the above conditions(funding_round_type)
master_frame_filtered = master_frame[master_frame.funding_round_type == most_suitable.index[0]]
master_frame_filtered.head(3)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0


### Table 3.1 ( Analysing the Top 3 English-Speaking Countries)
#### Q1/2/3. Top/Second/Third English speaking country ?

In [18]:
top9 = master_frame_filtered.groupby('country_code')['raised_amount_usd'].sum().sort_values(ascending=False)[0:9]
# combined top9.index[0:3]
top = top9.index[0]
second = top9.index[1]
third = top9.index[2]
print('top ',top)
print('second ',second)
print('third ',third)

top  USA
second  CHN
third  GBR


In [19]:
# Filter companies dataset based on top 3 english speaking countries
master_frame_filtered = master_frame_filtered.loc[master_frame_filtered.country_code.isin(top9.index[0:3])]
master_frame_filtered.head(3)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
5,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN


### Table 3.1 ( Sector-wise Investment Analysis)

In [20]:
# first string before the | will be considered the primary sector.
# Add the new column in companies_filtered dataframe named as 'primary_sector'
master_frame_filtered['primary_sector'] = master_frame_filtered['category_list'].str.split('|').str.get(0)
master_frame_filtered.head(3)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,Application Platforms
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,Curated Web
5,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN,Software


In [22]:
mapping =  pd.read_csv(dirpath+file3,encoding='iso-8859-1') # Use 'iso-8859-1' for accented characters
mapping.head(3)

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0


In [23]:
# Defining function demap, which will convert data in mapping dataframe above to two columns output.
def demap(df):
    l = list()
    c = int(df.count().sort_values(ascending=False)[0])
    for i in range(c):
        for col_name in df.columns:
            if df[col_name][i]==1:
               l.append(col_name)
    df2 = df.copy()
    df2['final_value'] = l
    return df2.iloc[:,[0,-1]]

In [24]:
mapping_new = demap(mapping)
mapping_new.head()

,category_list,final_value
0,NaN,Blanks
1,3D,Manufacturing
2,3D Printing,Manufacturing
3,3D Technology,Manufacturing
4,Accounting,"Social, Finance, Analytics, Advertising"


In [25]:
master_frame_sector_wise = pd.merge(left=master_frame_filtered, right=mapping_new,left_on='primary_sector',right_on='category_list')
# Drop unnecessary columns
master_frame_sector_wise.drop(labels=['category_list_x', 'category_list_y'],axis=1, inplace=True)
# Rename Columns
master_frame_sector_wise.rename(columns={'final_value':'main_sector'}, inplace=True)

In [27]:
master_frame_sector_wise.head(3)

,permalink,name,homepage_url,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector,main_sector
0,/organization/-qounter,:Qounter,http://www.qounter.com,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,Application Platforms,"News, Search and Messaging"
1,/organization/91app,91App,http://91app.com,operating,CHN,4,Taiwan,Taiwan,NaN,/organization/91app,/funding-round/e5b37ac80448e0e06273506eafd79a0e,venture,A,24-03-2015,9000000.0,Application Platforms,"News, Search and Messaging"
2,/organization/aarohi-communication,Aarohi communication,http://www.aarohi.net/,acquired,USA,CA,SF Bay Area,San Jose,NaN,/organization/aarohi-communication,/funding-round/1860d75993627f37c4b1a17b97a83502,venture,NaN,01-08-2001,NaN,Application Platforms,"News, Search and Messaging"


Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. 
The three data frames should contain:
All the columns of the master_frame along with the primary sector and the main sector
The total number (or count) of investments for each main sector in a separate column
The total amount invested in each main sector in a separate column

In [28]:
#USA
D1 = master_frame_sector_wise[master_frame_sector_wise['country_code']==top]

#TUR
D2 = master_frame_sector_wise[master_frame_sector_wise['country_code']==second]

#SWE
D3 = master_frame_sector_wise[master_frame_sector_wise['country_code']==third]

#### Q1. Total number of Investments (count)

In [29]:
print(D1.count()[0])
print(D2.count()[0])
print(D3.count()[0])

35759
1995
2158


 #### Q2. Total amount of investment (USD)

In [30]:
print(D1.raised_amount_usd.sum())
print(D2.raised_amount_usd.sum())
print(D3.raised_amount_usd.sum())

394680394849.0
37527936127.0
19097204731.0


#### Q3. Top sector (based on count of investments)

In [69]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0],'\n')

Others 

Others 

Others 



#### Q4. Second-best sector (based on count of investments)

In [48]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[1],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[1],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[1],'\n')

Cleantech / Semiconductors 

News, Search and Messaging 

Cleantech / Semiconductors 



#### Q5. Third-best sector (based on count of investments)

In [49]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[2],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[2],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[2],'\n')

Social, Finance, Analytics, Advertising 

Social, Finance, Analytics, Advertising 

Social, Finance, Analytics, Advertising 



#### Q6. Number of investments in the top sector (refer to point 3)

In [50]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[0],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[0],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[0],'\n')

8693 

613 

571 



#### Q7. Number of investments in the second-best sector (refer to point 4)

In [51]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[1],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[1],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[1],'\n')

8113 

330 

456 



#### Q8. Number of investments in the third-best sector (refer to point 5)

In [52]:
print(D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[2],'\n')
print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[2],'\n')
print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False)[2],'\n')

5553 

303 

359 



#### Q9. For the top sector count-wise (point 3), which company received the highest investment?

In [68]:
#print(D1.groupby(by='main_sector')['main_sector','permalink'].count().sort_values(ascending=False).index[0],'\n')
#print(D2.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0],'\n')
#print(D3.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0],'\n')

D1.groupby(by='main_sector')[['permalink']].count()
D1[D1.main_sector == D1.groupby(by='main_sector')['main_sector'].count().sort_values(ascending=False).index[0]]

,permalink,name,homepage_url,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector,main_sector
1259,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN,Software,Others
1260,/organization/1010data,1010data,http://www.1010data.com,acquired,USA,NY,New York City,New York,01-01-2000,/organization/1010data,/funding-round/482fbf992b48f5c47d0080ab7ec0cb54,venture,NaN,08-03-2010,35000000.0,Software,Others
1261,/organization/16-mile-solutions,16 Mile Solutions,NaN,operating,USA,WA,Seattle,Seattle,01-01-2006,/organization/16-mile-solutions,/funding-round/3a671cf6f08fe5d0a878f5627134384c,venture,NaN,11-06-2009,199957.0,Software,Others
1262,/organization/170-systems,170 Systems,http://www.170systems.com,acquired,USA,MA,Boston,Bedford,01-01-1990,/organization/170-systems,/funding-round/b84bb882ca873f5fb96535671981196d,venture,A,16-04-2002,14000000.0,Software,Others
1263,/organization/1energy-systems,1Energy Systems,http://1energysystems.com,operating,USA,WA,Seattle,Seattle,01-01-2010,/organization/1energy-systems,/funding-round/0502de190cb6fd03ba9435767080bcdb,venture,NaN,22-04-2013,1450000.0,Software,Others
1269,/organization/38-zeros,38 Zeros,http://www.38zeros.com/,operating,USA,OR,"Portland, Oregon",Portland,NaN,/organization/38-zeros,/funding-round/d59b46682e47e89673c218a36f290b81,venture,NaN,11-03-2015,800000.0,Software,Others
1270,/organization/3d-sports-technology,3D Sports Technology,http://3dsportstech.com,operating,USA,MN,Minneapolis,Minneapolis,01-01-2010,/organization/3d-sports-technology,/funding-round/8edeef5a343fd5fa6bd47dc317d92354,venture,NaN,21-01-2011,505000.0,Software,Others
1271,/organization/3d-sports-technology,3D Sports Technology,http://3dsportstech.com,operating,USA,MN,Minneapolis,Minneapolis,01-01-2010,/organization/3d-sports-technology,/funding-round/ac7a75f6b8b83d48056ecdf8efb0e872,venture,NaN,07-06-2012,110000.0,Software,Others
1272,/organization/3d-sports-technology,3D Sports Technology,http://3dsportstech.com,operating,USA,MN,Minneapolis,Minneapolis,01-01-2010,/organization/3d-sports-technology,/funding-round/e4cc7f5519c4556d7ee84c1b14d3a3ae,venture,NaN,25-07-2013,294940.0,Software,Others
1273,/organization/3d-systems,3D Systems,http://www.3dsystems.com,ipo,USA,SC,SC - Other,Rock Hill,01-01-1986,/organization/3d-systems,/funding-round/fa1d366d6517e4ca57a61dffce914067,venture,NaN,16-04-2014,17500000.0,Software,Others


#### Q10. For the second-best sector count-wise (point 4), which company received the highest investment?